<a href="https://colab.research.google.com/github/khanabdulmajid/tensorflow_deep_learning/blob/main/06__Transfer_learning_in_tensorflow_part_3_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#transfer learning with Tensorflow part 3: scaling up

we have seen the power if transfer learning feature extraction and fine tuning , now its time to scale up to all of the classes in food 101(101 toal classes)
Our goal is to beat the original food101 paper with 10% of the training data (leverageing the power of deep learning).

In [8]:
# !wget https://github.com/khanabdulmajid/tensorflow_deep_learning/a90154efbfd5eaf7734bf1e073a94b98d09946dd/helper_functions.py
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2023-09-02 10:57:25--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-09-02 10:57:25 (117 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [9]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [11]:
unzip_data("/content/10_food_classes_10_percent.zip")

In [28]:
checkpoint_path="/content/modelcheckpoint"
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                       monitor="val_accuracy",
                                                       save_best_only=True,
                                                       save_weights_only=True)
train_dir="/content/10_food_classes_10_percent/train"
test_dir="/content/10_food_classes_10_percent/test"

In [13]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import preprocessing

In [30]:
IMG_SIZE=(224,224)
train_data_10_percent=preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="categorical",
                                                                 image_size=IMG_SIZE
                                                                 )
test_data=preprocessing.image_dataset_from_directory(test_dir,
                                                     label_mode="categorical",
                                                     image_size=IMG_SIZE
                                                     )

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [35]:
data_augmentation=Sequential([
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.2)
],  name="data_augmentation")

base_model =tf.keras.applications.EfficientNetB0(include_top=False)

In [33]:
base_model.trainable=False

inputs=tf.keras.layers.InputLayer(input_shape=(224,224))
x = data_augmentation(inputs)
x = base_model(x,trainable=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs=tf.keras.layers.Dense(10,activation="softmax")(x)
model_0=tf.keras.Model(inputs,outputs)

ValueError: ignored